In [ ]:
from flask import Flask
from flask import request
import nltk.stem.porter as stemmer
import json
import requests

app = Flask(__name__)


@app.route("/analyze", methods=["POST"])
def analyze():
    j = request.json
    query = j['data']
    words = query.split(' ')
    #stemming
    response_stemmed = requests.post('http://127.0.0.1:13535/analysis/stemmer', json={'data' : words})
    parsed_stemmed = json.loads(response_stemmed.text)
    
    #delete stopwords
    response_stopwords = requests.post('http://127.0.0.1:13536/analysis/stopwords', json={'data' : parsed_stemmed['processed_data']})
    parsed_stopwords = json.loads(response_stopwords.text)
    
    #add synonymus
    response_synonymus = requests.post('http://127.0.0.1:13539/analysis/synonymus', json={'data' : parsed_stopwords['processed_data']})
    parsed_synonymus = json.loads(response_synonymus.text)
    
    #fix misprints
    response_fixed = requests.post('http://127.0.0.1:13540/analysis/misprints', json={'data' : parsed_stopwords['processed_data']})
    parsed_fixed_text = json.loads(response_fixed.text)
    return json.dumps({"status":"ok", "words": parsed_fixed_text['processed_data'], "language" : parsed_stemmed['language']})
    

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=13533)

 * Running on http://0.0.0.0:13533/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Nov/2018 13:17:48] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2018 13:18:59] "POST /analyze HTTP/1.1" 200 -
127.0.0.1 - - [07/Nov/2018 13:20:50] "POST /analyze HTTP/1.1" 200 -
